<a href="https://colab.research.google.com/github/buthaina279/LSTM_MIMIC-III/blob/main/prepare_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#for google drive
DATA_FILEPATH = "/content/drive/MyDrive/final_project/all_hourly_data.h5"

In [ ]:
import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss

Read the data of the patients

In [ ]:
statics = pd.read_hdf(DATA_FILEPATH, 'patients')

In [ ]:
statics.head()

,,,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,discharge_location,fullcode_first,dnr_first,...,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,SNF,1.0,0.0,...,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,0,0,0,1,0,145
4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,HOME WITH HOME IV PROVIDR,1.0,0.0,...,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,0,0,0,1,0,40
6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,HOME HEALTH CARE,1.0,0.0,...,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,0,0,0,1,0,88
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,DEAD/EXPIRED,1.0,0.0,...,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,1,1,1,1,0,127
11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,HOME HEALTH CARE,1.0,0.0,...,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,0,0,0,1,0,38


In [ ]:
statics.shape

(34472, 28)

Check how many levels for the multiIndex

In [ ]:
nblevels = statics.index.nlevels
print("the number of levels are", nblevels)

the number of levels are 3


Reset index of the DataFrame

In [ ]:
#1
df = statics.reset_index()


In [ ]:
df.head()

,subject_id,hadm_id,icustay_id,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,...,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours
0,3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,...,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,0,0,0,1,0,145
1,4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,...,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,0,0,0,1,0,40
2,6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,...,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,0,0,0,1,0,88
3,9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,...,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,1,1,1,1,0,127
4,11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,...,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,0,0,0,1,0,38


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34472 entries, 0 to 34471
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   subject_id              34472 non-null  int64         
 1   hadm_id                 34472 non-null  int64         
 2   icustay_id              34472 non-null  int64         
 3   gender                  34472 non-null  category      
 4   ethnicity               34472 non-null  category      
 5   age                     34472 non-null  float64       
 6   insurance               34472 non-null  object        
 7   admittime               34472 non-null  datetime64[ns]
 8   diagnosis_at_admission  34471 non-null  object        
 9   dischtime               34472 non-null  datetime64[ns]
 10  discharge_location      34472 non-null  object        
 11  fullcode_first          28162 non-null  float64       
 12  dnr_first               28162 non-null  float6

In [ ]:
df.shape

(34472, 31)

In [ ]:
df.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'ethnicity', 'age',
       'insurance', 'admittime', 'diagnosis_at_admission', 'dischtime',
       'discharge_location', 'fullcode_first', 'dnr_first', 'fullcode', 'dnr',
       'dnr_first_charttime', 'cmo_first', 'cmo_last', 'cmo', 'deathtime',
       'intime', 'outtime', 'los_icu', 'admission_type', 'first_careunit',
       'mort_icu', 'mort_hosp', 'hospital_expire_flag', 'hospstay_seq',
       'readmission_30', 'max_hours'],
      dtype='object')

Drop times columns, not in ICU mortality, and text columns

In [ ]:
df.drop(['admittime', 'dischtime','deathtime', 'intime', 'outtime', 'dnr_first_charttime'
, 'diagnosis_at_admission', 'discharge_location'
#, 'fullcode_first', 'dnr_first', 'fullcode', 'dnr'
#, 'cmo_first', 'cmo_last', 'cmo'
,'mort_hosp', 'hospital_expire_flag', ], axis=1, inplace=True)

In [ ]:
df.head()

,subject_id,hadm_id,icustay_id,gender,ethnicity,age,insurance,fullcode_first,dnr_first,fullcode,...,cmo_first,cmo_last,cmo,los_icu,admission_type,first_careunit,mort_icu,hospstay_seq,readmission_30,max_hours
0,3,145834,211552,M,WHITE,76.526792,Medicare,1.0,0.0,1.0,...,0.0,0.0,0.0,6.064560,EMERGENCY,MICU,0,1,0,145
1,4,185777,294638,F,WHITE,47.845047,Private,1.0,0.0,1.0,...,0.0,0.0,0.0,1.678472,EMERGENCY,MICU,0,1,0,40
2,6,107064,228232,F,WHITE,65.942297,Medicare,1.0,0.0,1.0,...,0.0,0.0,0.0,3.672917,ELECTIVE,SICU,0,1,0,88
3,9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,1.0,0.0,1.0,...,0.0,0.0,0.0,5.323056,EMERGENCY,MICU,1,1,0,127
4,11,194540,229441,F,WHITE,50.148295,Private,1.0,0.0,1.0,...,0.0,0.0,0.0,1.584410,EMERGENCY,SICU,0,1,0,38


In [ ]:
df.shape

(34472, 21)

Summary of statistics pertaining to the DataFrame columns with ignoring the null values

In [ ]:
df.describe()

,subject_id,hadm_id,icustay_id,age,fullcode_first,dnr_first,fullcode,dnr,cmo_first,cmo_last,cmo,los_icu,mort_icu,hospstay_seq,readmission_30,max_hours
count,34472.000000,34472.000000,34472.000000,34472.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,34472.000000,34472.000000,34472.0,34472.000000,34472.000000
mean,38749.629990,149995.737497,250196.326584,74.554102,0.936297,0.058270,0.939955,0.110752,0.003054,0.032526,0.034017,2.639321,0.065589,1.0,0.043223,62.847586
std,29451.376793,28961.212190,28931.600839,55.958878,0.244227,0.234257,0.237575,0.313830,0.055177,0.177396,0.181277,1.982919,0.247567,0.0,0.203363,47.591847
min,3.000000,100001.000000,200003.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,1.0,0.000000,12.000000
25%,14226.500000,124855.250000,225206.500000,52.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.167674,0.000000,1.0,0.000000,28.000000
50%,28363.500000,150060.500000,250233.000000,65.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.981950,0.000000,1.0,0.000000,47.000000
75%,63663.000000,175152.500000,275278.250000,78.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.310964,0.000000,1.0,0.000000,79.000000
max,99999.000000,199999.000000,299999.000000,310.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.999456,1.000000,1.0,1.000000,239.000000


Check the percentage of the missing values

In [ ]:
def missing(df):
        # Total missing values
        val = df.isnull().sum()
        
        # Percentage of missing values
        percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        table = pd.concat([val, percent], axis=1)
        
        # Rename the columns
        table_ren_columns = table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        table_ren_columns = table_ren_columns[table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
        # Return the dataframe with missing information
        return table_ren_columns

In [ ]:
missing(df)

,Missing Values,% of Total Values
fullcode_first,6310,18.3
dnr_first,6310,18.3
fullcode,6310,18.3
dnr,6310,18.3
cmo_first,6310,18.3
cmo_last,6310,18.3
cmo,6310,18.3


Check the number of ICU stay and patients' IDs

In [ ]:
IDs = list(set(df['icustay_id'].tolist())) # get all the time series ID's
num_samples = len(IDs)
print(num_samples)

34472


In [ ]:
IDs = list(set(df['subject_id'].tolist())) # get all the time series ID's
num_samples = len(IDs)
print(num_samples)

34472


In [ ]:
df.to_csv('/content/drive/MyDrive/final_project/statistics.csv') 